## 📊 Notebook 2: Análisis exploratorio de datos (EDA)
En esta notebook se lleva a cabo un análisis exploratorio de las reseñas procesadas. Se examina la distribución de calificaciones, la evolución temporal de los comentarios, patrones de longitud en los textos y otras variables relevantes. Este paso permite obtener una primera comprensión de la estructura de los datos y guiar futuras decisiones analíticas.


### 📥 Carga de datos procesados

En esta sección se carga el dataset previamente limpiado y unificado durante la etapa de preprocesamiento. El archivo incluye reseñas de productos (Samsung A15 y Motorola G32), junto con sus calificaciones, fechas y textos normalizados. Esto permitirá realizar un análisis exploratorio confiable sobre los patrones de opinión de los usuarios.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo base
sns.set_theme(style="darkgrid")


In [ ]:
# Cargar datos limpios desde la etapa de preprocesamiento
df_total = pd.read_csv("../data/processed/reviews_unificado.csv", parse_dates=['date'])

# Vista general del DataFrame
df_total.info()
df_total.head()

# 1. Volumen mensual y acumulado de reseñas publicadas

### 🗓️ Distribución temporal de las reseñas

Visualización de la cantidad de reseñas a lo largo del tiempo. Esto permite detectar tendencias o picos de interés por los productos.

In [ ]:
df_total['year_month'] = df_total['date'].dt.to_period('M')

df_total['producto'] = df_total['producto'].astype('category')

reviews_by_month_product = df_total.groupby(['year_month', 'producto'], observed=False).size().reset_index(name='count')

sns.set_style("darkgrid")
plt.figure(figsize=(12, 6))
sns.barplot(x='year_month', y='count', hue='producto', data=reviews_by_month_product)
plt.title("Cantidad de reseñas por mes, discriminado por producto")
plt.xlabel("Fecha")
plt.ylabel("Número de reseñas")
plt.xticks(rotation=45)
plt.tight_layout()

plt.savefig("../outputs/visualizations/01_grafico_cantidad_de_reseñas.png", dpi=300, bbox_inches='tight')

plt.show()


### Curva acumulativa de reseñas por producto
La curva acumulativa permite analizar el ritmo de adopción y el ciclo de vida de las reseñas, reflejando la evolución de la presencia del producto en el mercado digital.



In [ ]:
# Crear el DataFrame acumulativo
df_cumulative = df_total.groupby(['producto', 'year_month'], observed=False).size().groupby(level=0, observed=False).cumsum().reset_index(name='acumuladas')

# Convertir year_month de Period a datetime usando to_timestamp()
df_cumulative['year_month'] = df_cumulative['year_month'].dt.to_timestamp()

plt.figure(figsize=(14, 7))

sns.lineplot(data=df_cumulative, x='year_month', y='acumuladas', hue='producto', marker='o')

plt.title('Curva acumulativa de reseñas por producto')
plt.ylabel('Total acumulado de reseñas')
plt.xlabel('Mes')
plt.xticks(rotation=45)
plt.savefig("../outputs/visualizations/01_gráfico_acumulativo_de_reseñas.png", dpi=300, bbox_inches='tight')
plt.show()

# 2. Distribución y evolución temporal de las calificaciones

### 📊 Proporción de ratings por producto

Este gráfico de barras muestra la distribución relativa de calificaciones (ratings) dentro de cada producto. A diferencia de los conteos absolutos, aquí cada barra representa la proporción de reseñas con determinado puntaje sobre el total de reseñas de ese producto, lo cual permite una comparación más justa entre productos con diferente volumen de opiniones.

In [ ]:

rating_proportions = (
    df_total
    .groupby('producto', observed=False)['rating']
    .value_counts(normalize=True)
    .rename('proportion')
    .reset_index()
)

sns.barplot(
    data=rating_proportions,
    x='rating',
    y='proportion',
    hue='producto',
)

plt.title("Distribución proporcional de ratings por producto")
plt.xlabel("Rating")
plt.ylabel("Proporción dentro del producto")
plt.legend(title='Producto')
plt.tight_layout()

plt.savefig("../outputs/visualizations/01_grafico_ratings_por_productos.png", dpi=300, bbox_inches='tight')
plt.show()


### 📊 Distribución de calificaciones a lo largo del tiempo

Este gráfico muestra la evolución mensual de la calificación promedio otorgada por los usuarios para cada producto. Permite identificar posibles tendencias en la percepción del producto, como mejoras o deterioros en la experiencia de uso reportada. También puede reflejar cambios en el tipo de comprador o en las expectativas a lo largo del tiempo.

- Se calcula el promedio de `rating` por mes (`date` agregada a nivel de `month`) y por producto.
- Las líneas permiten visualizar comparativamente cómo se comporta la valoración de cada producto en el tiempo.
- El eje Y está restringido entre 1 y 5, que son los valores posibles de calificación en la plataforma.


In [ ]:
calificacion_mensual = (
    df_total
    .assign(month=pd.to_datetime(df_total['date'].dt.to_period('M').astype(str)))
    .groupby(['month', 'producto'], observed=True)['rating']
    .mean()
    .reset_index()
)

# Configuración del estilo y gráfico
sns.set_style("darkgrid")  # Fondo con cuadrícula oscura
plt.figure(figsize=(12, 6))

# Gráfico de líneas con marcadores
sns.lineplot(
    data=calificacion_mensual, 
    x='month', 
    y='rating',  # <<-- Corregido: 'rating' en lugar de 'rating_proportions'
    hue='producto', 
    marker='o',  # Marcadores circulares
    linewidth=2.5  # Grosor de línea aumentado para mejor visibilidad
)

# Personalización del gráfico
plt.title("Evolución mensual de la calificación promedio", fontsize=14, pad=20)
plt.xlabel("Mes", fontsize=12)
plt.ylabel("Calificación promedio", fontsize=12)
plt.xticks(rotation=45)
plt.ylim(1, 5.2)  # Margen superior mejorado
#plt.legend(title='Producto', bbox_to_anchor=(1.05, 1), loc='upper left')  # Leyenda fuera del gráfico

plt.tight_layout()  # Ajuste automático para evitar cortes
plt.savefig("../outputs/visualizations/01_gráfico_evolución_calificación_promedio.png", dpi=300, bbox_inches='tight')
plt.show()

### Balance mensual de ratings positivos (4-5) vs negativos (1-2)
Este gráfico apilado permite visualizar mes a mes la proporción relativa de reseñas positivas frente a negativas dentro del volumen total del producto. Resulta útil para identificar ciclos de saturación reputacional o momentos críticos.

In [ ]:
sns.set_style("darkgrid")

paletas = {
    'Producto1': {'Positivo (4-5)': 'cadetblue', 'Negativo (1-2)': 'red'},
    'Producto2': {'Positivo (4-5)': 'cadetblue', 'Negativo (1-2)': 'red'}
}

def calcular_proporciones(df):
    conteos = df.groupby(['year_month', 'producto', 'rating'], observed=False).size().reset_index(name='count')
    conteos['proportion'] = conteos['count'] / conteos.groupby(['year_month', 'producto'], observed=False)['count'].transform('sum')
    return conteos

df_proporciones = calcular_proporciones(df_total)
productos_ordenados = sorted(df_proporciones['producto'].unique())

for i, producto in enumerate(productos_ordenados):
    df_producto = df_proporciones[df_proporciones['producto'] == producto].copy()
    
    df_producto['rating_cat'] = np.where(
        df_producto['rating'] >= 4, 'Positivo (4-5)',
        np.where(df_producto['rating'] <= 2, 'Negativo (1-2)', 'Neutral (3)')
    )
    
    df_pivot = (
        df_producto.groupby(['year_month', 'rating_cat'], observed=False)['proportion'].sum()
        .unstack(fill_value=0)
        .reindex(columns=['Positivo (4-5)', 'Negativo (1-2)'], fill_value=0)
    )
    
    # Filtrar meses sin datos
    df_pivot = df_pivot[(df_pivot['Positivo (4-5)'] != 0) | (df_pivot['Negativo (1-2)'] != 0)]
    
    fig, ax = plt.subplots(figsize=(10, 4))
    paleta = paletas[f'Producto{i+1}'] if i < len(paletas) else paletas['Producto1']
    
    df_pivot.plot(
        kind='bar',
        stacked=True,
        color=[paleta['Positivo (4-5)'], paleta['Negativo (1-2)']],
        alpha=1,
        ax=ax
    )
    
    ax.set_title(f"Balance mensual de ratings - {producto}", pad=20, fontsize=14, fontweight='bold', loc='center')
    ax.set_ylabel('Proporción', fontsize=12)
    ax.set_xlabel('Mes', fontsize=12)
    ax.grid(axis='y', color='lightgray', linestyle='--', alpha=0.4)
    ax.set_xlim(-0.5, len(df_pivot)-0.5)
    
    legend = ax.legend(
        #title='Rating',
        frameon=True,
        facecolor='white',
        edgecolor='gray',
        bbox_to_anchor=(0.87, 1.24),
        loc='upper left',
        borderaxespad=0.5,
        fontsize=9,
        ncol=1
    )
    legend.get_title().set_fontsize(12)
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Guardar cada gráfico con nombre descriptivo
    nombre_archivo = f"../outputs/visualizations/01_balance_ratings_mensual_{producto.lower().replace(' ', '_')}.png"
    plt.savefig(nombre_archivo, dpi=300, bbox_inches='tight')
    
    plt.show()

## 2. Análisis de la longitud de la reseñas

### 📊 a. Distribución de longitud de reseñas por producto

**Presentación**

En esta sección se analiza la **cantidad de palabras** utilizadas por los usuarios al redactar sus reseñas, comparando los modelos **Motorola G32** y **Samsung A15**. Se calcularon **estadísticas descriptivas** (promedio, mediana, desviación estándar, percentiles, outliers) y se visualizó la distribución mediante un **boxplot por producto**.

El objetivo es detectar **diferencias en la extensión de los comentarios**, lo cual puede reflejar distintos niveles de **detalle, involucramiento o satisfacción** según el dispositivo.

In [ ]:
# Calcular estadísticas descriptivas agrupadas por producto
stats_producto = df_total.groupby('producto')['text_length'].describe(percentiles=[.25, .5, .75])
stats_producto = stats_producto.rename(columns={'50%': 'mediana'})

# Calcular IQR y límites de outliers
stats_producto['IQR'] = stats_producto['75%'] - stats_producto['25%']
stats_producto['Límite inferior (outliers)'] = stats_producto['25%'] - 1.5 * stats_producto['IQR']
stats_producto['Límite superior (outliers)'] = stats_producto['75%'] + 1.5 * stats_producto['IQR']

# Reordenar columnas para mejor presentación
column_order = [
    'count', 'mean', 'std', 'min', '25%', 'mediana', '75%', 'max', 'IQR',
    'Límite inferior (outliers)', 'Límite superior (outliers)'
]
stats_producto = stats_producto[column_order]

# Mostrar resultados
print("Estadísticas de longitud de reseñas por producto:")
display(stats_producto.round(2))

# Opcional: Guardar en CSV
stats_producto.to_csv('estadisticas_longitud_por_producto.csv', encoding='utf-8-sig')

In [ ]:
sns.set_theme(style="darkgrid") 
sns.boxplot(data=df_total, x='producto', y='text_length', hue='producto', legend=False)
plt.title("Distribución de longitud de reseñas por producto")
plt.xlabel("Producto")
plt.ylabel("Cantidad de palabras por reseña")

plt.savefig("../outputs/visualizations/01_grafico_longitud_reseñas_por_productos.png", dpi=300, bbox_inches='tight')

plt.show()

### 🔍 Análisis e interpretación

Ambos productos muestran distribuciones similares, aunque con **diferencias notables en dispersión y valores extremos**:

- **Motorola G32**  
  - Promedio: **19.03 palabras**  
  - Mediana: **12 palabras**  
  - IQR: **18 palabras**  
  - Límite para outliers: **50 palabras**  
  - Máximo observado: **198 palabras**

- **Samsung A15**  
  - Promedio: **16.58 palabras**  
  - Mediana: **12 palabras**  
  - IQR: **15 palabras**  
  - Límite para outliers: **43.5 palabras**  
  - Máximo observado: **181 palabras**

Ambos modelos comparten una **mediana idéntica** (12 palabras), pero el **Motorola G32** presenta una **mayor dispersión** y más **outliers superiores**, lo que indica una mayor tendencia a redactar reseñas extensas en casos particulares.

El **boxplot** refleja esta diferencia: mayor altura de la caja y más valores atípicos en el Motorola G32, frente a una distribución más **compacta y homogénea** en el Samsung A15.

Estas variaciones podrían vincularse a factores como la **trayectoria comercial**, el **perfil del usuario** o las **expectativas** en torno al producto. En términos generales, el Motorola G32 muestra un patrón **ligeramente más expansivo** en la redacción de reseñas.


#### b. Distribución de longitud de reseñas según calificación

En esta sección se analiza la relación entre la calificación asignada por los usuarios (de 1 a 5 estrellas) y la longitud de las reseñas que redactan, medida en cantidad de palabras. Para ello, se generan estadísticas descriptivas agrupadas por calificación (media, mediana, desviación estándar, percentiles, valores atípicos) y se visualiza la distribución mediante un boxplot.

El objetivo es identificar si existe una tendencia en la longitud del texto en función de la valoración asignada, lo cual puede ofrecer pistas sobre la intensidad emocional, la necesidad de argumentar o el nivel de detalle involucrado en cada tipo de reseña.


In [ ]:

# Calcular estadísticas por grupo de calificación
stats = df_total.groupby('rating')['text_length'].describe(percentiles=[.25, .5, .75])
stats = stats.rename(columns={'50%': 'mediana'})  # Renombrar la mediana

# Calcular el rango intercuartílico (IQR) y límites de outliers
stats['IQR'] = stats['75%'] - stats['25%']
stats['Límite inferior (outliers)'] = stats['25%'] - 1.5 * stats['IQR']
stats['Límite superior (outliers)'] = stats['75%'] + 1.5 * stats['IQR']

# Reordenar columnas para mejor presentación
stats = stats[['count', 'mean', 'std', 'min', '25%', 'mediana', '75%', 'max', 'IQR', 
               'Límite inferior (outliers)', 'Límite superior (outliers)']]

# Mostrar resultados
display(stats.round(2))

In [ ]:
plt.figure(figsize=(10, 6))

# Crear el boxplot asegurando el orden de las calificaciones
sns.boxplot(data=df_total, 
            x='rating',  # Especificamos rating en el eje x
            y='text_length', 
            order=sorted(df_total['rating'].unique()),  # Ordenamos las calificaciones
            palette='hls')

# Títulos y etiquetas mejoradas
plt.title("Distribución de longitud de reseñas por calificación", pad=20, fontsize=14)
plt.xlabel("Calificación (1-5)", fontsize=12)
plt.ylabel("Cantidad de palabras en reseña limpia", fontsize=12)

# Ajustar los ticks del eje x para mostrar todas las calificaciones
plt.xticks(ticks=range(5), labels=[1, 2, 3, 4, 5])  # Fuerza a mostrar todos los valores

# Mejorar la presentación
sns.despine()  # Eliminar bordes superfluos
plt.tight_layout()

# Guardar la figura
plt.savefig("../outputs/visualizations/01_grafico_distribución_longitud_reseñas_por_calificacion.png", 
            dpi=300, 
            bbox_inches='tight')

plt.show()

#### Análisis e interpretación

Los resultados indican que las reseñas con calificaciones intermedias-bajas (2 y 3 estrellas) tienden a ser más extensas en promedio, con medias superiores a 19 palabras y los rangos intercuartílicos más amplios del conjunto (19 y 17.5 palabras, respectivamente). Este patrón sugiere que, ante una experiencia ambigua o parcialmente insatisfactoria, los usuarios sienten la necesidad de justificar o matizar su puntuación mediante un discurso más elaborado.

Por el contrario, las reseñas con calificación 1 —aunque negativas— tienden a ser más breves, lo que podría estar relacionado con una descarga emocional más inmediata o lapidaria. Las calificaciones más altas (4 y 5 estrellas) presentan tanto menores medias (alrededor de 17 palabras o menos) como una distribución más concentrada, lo que refleja formas de evaluación más directas, resumidas y estandarizadas.

En el boxplot, esta diferencia se hace visible tanto en la altura de las cajas como en la dispersión de los valores atípicos. Las reseñas de 2 y 3 estrellas sobresalen por su mayor elongación vertical y la presencia de comentarios largos, mientras que las puntuaciones extremas (1 y 5) muestran distribuciones más compactas.

En conjunto, los datos sugieren que la longitud de las reseñas no se relaciona linealmente con la intensidad de la calificación, sino que responde a distintos estilos discursivos: mientras que el descontento moderado estimula la argumentación, la insatisfacción total y la aprobación entusiasta tienden a expresarse en registros más sintéticos.


### Evolución mensual de la longitud de reseñas

En esta sección se analiza cómo ha cambiado a lo largo del tiempo la longitud de los comentarios dejados por los usuarios para los modelos Motorola G32 y Samsung A15, medida en número de palabras por reseña.

#### 1. Presentación del análisis

A continuación se expone un análisis comparado de la longitud de las reseñas en ambos modelos, abordado desde tres ángulos:

- 📊 Estadísticas descriptivas mensuales: media, mediana, desviación estándar y valores extremos.
- 📈 Visualización de tendencias: evolución de la longitud promedio por mes.
- 📦 Dispersión e outliers: distribución mensual mediante boxplots.

Este enfoque busca identificar diferencias en el comportamiento discursivo de los usuarios, detectar patrones de regularidad o cambio a lo largo del tiempo y evaluar posibles eventos que hayan incidido en la extensión de los comentarios.

#### 2. Generación de datos y visualizaciones

Las siguientes celdas contienen el procesamiento de los datos, el cálculo de las estadísticas mensuales y la generación de dos gráficos:

- **Gráfico 1:** evolución mensual de la longitud promedio de las reseñas.
- **Gráfico 2:** boxplots mensuales que muestran la dispersión y presencia de outliers.

#### 3. Análisis interpretativo

Una vez presentados los datos y visualizaciones, se ofrece una interpretación cualitativa de los patrones observados, destacando diferencias entre los productos, evolución temporal y posibles explicaciones discursivas o contextuales.


### Calculo de estadisticas por me y producto

In [ ]:
import pandas as pd

# Calcular estadísticas agrupadas por mes y producto
stats_mensuales = df_total.groupby(['year_month', 'producto'])['text_length'].describe(percentiles=[.25, .5, .75])
stats_mensuales = stats_mensuales.rename(columns={'50%': 'mediana'})

# Calcular IQR y límites de outliers
stats_mensuales['IQR'] = stats_mensuales['75%'] - stats_mensuales['25%']
stats_mensuales['Límite inferior (outliers)'] = stats_mensuales['25%'] - 1.5 * stats_mensuales['IQR']
stats_mensuales['Límite superior (outliers)'] = stats_mensuales['75%'] + 1.5 * stats_mensuales['IQR']

# Reordenar columnas para mejor presentación
columnas_ordenadas = [
    'count', 'mean', 'std', 'min', '25%', 'mediana', '75%', 'max', 'IQR',
    'Límite inferior (outliers)', 'Límite superior (outliers)'
]
stats_mensuales = stats_mensuales[columnas_ordenadas]

# Resetear el índice para mejor visualización
stats_mensuales_reset = stats_mensuales.reset_index()

# Mostrar las primeras filas (opcional: guardar en CSV/Excel)
print(stats_mensuales_reset.head(10))

# Guardar en CSV
stats_mensuales_reset.to_csv('estadisticas_mensuales_por_producto.csv', index=False, encoding='utf-8-sig')

#### Grafico 1. Evolución mensual de la longitud promedio de reseñas

In [ ]:
longitud_mensual = (
    df_total
    .assign(month=pd.to_datetime(df_total['date'].dt.to_period('M').astype(str)))  # mes como datetime
    .groupby(['month', 'producto'], observed=True)['text_length']
    .mean()
    .reset_index()
)

plt.figure(figsize=(12, 6))
sns.lineplot(data=longitud_mensual, x='month', y='text_length', hue='producto', marker='o')
plt.title("Evolución mensual de la longitud promedio de reseñas")
plt.xlabel("Mes")
plt.ylabel("Longitud promedio (número de palabras)")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()

plt.savefig("../outputs/visualizations/01_grafico_evolucion_de_longitud_reseñas_por_productos.png", dpi=300, bbox_inches='tight')

plt.show()


#### Gráfico 2. Boxplot mensual de longitud de reseñas

In [ ]:
df_total = df_total.sort_values('year_month')

plt.figure(figsize=(14, 7))
sns.boxplot(data=df_total, x='year_month', y='text_length', hue='producto')
plt.title('Boxplot mensual de longitud de reseñas')
plt.ylabel('Número de palabras')
plt.xlabel('Mes')
plt.xticks(rotation=45)
plt.legend(title='Producto')

plt.savefig("../outputs/visualizations/01_grafico_boxplot_mensual_longitud_reseñas.png", dpi=300, bbox_inches='tight')
plt.show()

### Análisis Estadístico Descriptivo

#### 🔍 Diferencias entre productos

- **Motorola G32** mostró una **mayor variabilidad** en la longitud de las reseñas a lo largo del tiempo.  
  - El valor **máximo** de longitud alcanzó las **198 palabras** en **julio de 2023**, mes en el que también se registró el **mayor promedio mensual** (**media = 44.6 palabras**).
  - Otros picos destacados se observan en **abril de 2023** (**máximo = 102**) y **noviembre de 2023** (**máximo = 177**).

- **Samsung A15** presentó una **distribución más estable**, con una mediana mensual que osciló entre **2 y 15 palabras**.  
  - Aun así, se detectaron **outliers** significativos, con valores máximos por encima de 100 palabras en varios meses:
    - **septiembre 2024** (114)
    - **octubre 2024** (119)
    - **enero 2025** (181)
    - **marzo 2025** (104)

#### 📈 Tendencias temporales

- **Motorola G32** exhibió una **disminución progresiva** en la dispersión de la longitud de las reseñas:
  - El **IQR promedio** fue de **21.2 en 2023**, mientras que en **2024** descendió a **14.7**.
  - Esto indica una tendencia hacia **mayor homogeneidad** en las reseñas recientes (media del IQR total: **17.1**).

- **Samsung A15** mantuvo una **mediana mensual bastante estable** en torno a las **10–15 palabras**,  
  aunque con una presencia creciente de reseñas largas hacia finales de 2024 y principios de 2025.  
  - El punto más alto fue **enero de 2025**, con un máximo de **181 palabras**.

#### ⚠️ Outliers

Ambos productos presentan reseñas **extremadamente largas** (más de 100 palabras) en meses específicos:

- **Motorola G32**: abril 2023 (102), julio 2023 (198), noviembre 2023 (177), agosto 2024 (136).
- **Samsung A15**: septiembre 2024 (114), octubre 2024 (119), enero 2025 (181), marzo 2025 (104).

Estas reseñas atípicas podrían reflejar **momentos de fuerte involucramiento emocional** o eventos particulares que afectaron la experiencia del usuario.

---

### 2. Boxplot mensual por producto

El primer gráfico, correspondiente a los **boxplots mensuales de la longitud de las reseñas**, permite observar la **distribución**, **dispersión** y **presencia de outliers** para cada producto en el tiempo.

#### 📊 Motorola G32:

- Mayor dispersión en la **primera mitad de 2023**, especialmente entre abril y julio.
- Picos de longitud superiores a **100 palabras**, alcanzando un máximo de **198 palabras en julio 2023**.
- En estos meses, el **IQR** fue notablemente alto (**31.5 en julio 2023**), reflejando **alta heterogeneidad** en los comentarios.
- A partir de 2024, la dispersión disminuye: **boxplots más estrechos** y reducción de valores extremos → reseñas más uniformes.

#### 📊 Samsung A15:

- Desde su lanzamiento (**abril 2024**), presenta una **distribución más contenida**.
- Las **medianas** se sitúan mayoritariamente entre **8 y 13 palabras**.
- Se observan **outliers ocasionales** (más de 100 palabras), principalmente en:
  - **septiembre y octubre 2024**
  - **enero y marzo 2025**
- Comparado con el G32, muestra **menor varianza estructural** y una **curva de evolución más estable**.

---

### 3. Evolución de la longitud promedio mensual

El segundo gráfico muestra la evolución temporal de la **longitud promedio de las reseñas por mes**, facilitando la identificación de **tendencias generales** y **picos significativos**.

#### 📈 Motorola G32:

- Entre **abril y julio de 2023**, se observan picos con promedios **superiores a 37 palabras**, llegando a un máximo de **44.6 palabras en julio**.
- Desde **agosto 2023**, la media mensual **desciende gradualmente**.
- A partir de mediados de 2024, se estabiliza por debajo de **20 palabras**, lo que podría reflejar:
  - Agotamiento del entusiasmo inicial.
  - Homogeneización del tipo de usuario que deja comentarios.
  - Menor necesidad de argumentación detallada en las reseñas.

#### 📈 Samsung A15:

- Desde su aparición, mantiene una **tendencia regular** entre **15 y 25 palabras promedio**.
- El máximo mensual ocurre en **enero de 2025** con **25.5 palabras**, coincidiendo con el outlier de **181 palabras** visto en el boxplot.
- Su curva es más **plana y predecible**, lo que sugiere un comportamiento discursivo menos emocional o menos influido por eventos singulares.


# 3. Análisis de votos útiles

En esta sección examinamos los patrones de interacción con el sistema de votos a través de tres enfoques complementarios:

1. **Distribución general de votos útiles**: Análisis global de esta métrica (tendencia central, dispersión y valores atípicos) para entender el comportamiento base de los usuarios.

2. **Relación con calificaciones**: Investigamos cómo varían los votos según las estrellas asignadas (1-5), revelando posibles sesgos en la percepción de utilidad.

3. **Impacto de la extensión**: Analizamos si reseñas más largas reciben mayor engagement, comparando la distribución de votos útiles por rangos de longitud de texto.

Mediante estadísticas descriptivas y visualizaciones, buscamos identificar qué factores influyen en la utilidad percibida de las reseñas.

### a. Distribución general de votos útiles

In [ ]:
# Filtrar datos
df_filtrado = df_total[df_total['useful_votes'] <= 100]

# 1. Estadísticas descriptivas básicas por producto
stats_votos_utiles = df_filtrado.groupby('producto')['useful_votes'].describe(percentiles=[.25, .5, .75, .95])
stats_votos_utiles = stats_votos_utiles.rename(columns={'50%': 'mediana'})

# 2. Estadísticas generales (totales)
stats_totales = df_filtrado['useful_votes'].describe(percentiles=[.25, .5, .75, .95]).to_frame().T
stats_totales = stats_totales.rename(columns={'50%': 'mediana'})
stats_totales.index = ['TOTAL']

# 3. Porcentaje de reseñas con votos útiles > 0
def calcular_porcentaje_votos(group):
    return (group > 0).mean() * 100

resenas_con_votos = (
    df_filtrado.groupby('producto')['useful_votes']
    .apply(calcular_porcentaje_votos)
    .rename('% reseñas con votos útiles')
)

resenas_con_votos_total = pd.Series(
    [calcular_porcentaje_votos(df_filtrado['useful_votes'])],
    index=['TOTAL'],
    name='% reseñas con votos útiles'
)

# 4. Unir todas las tablas
stats_completas = pd.concat([
    pd.concat([stats_votos_utiles, stats_totales]),
    pd.concat([resenas_con_votos, resenas_con_votos_total])
], axis=1)

# 5. Ordenar columnas y formatear
column_order = [
    'count', '% reseñas con votos útiles', 'mean', 'std', 'min', '25%', 
    'mediana', '75%', '95%', 'max'
]
stats_completas = stats_completas[column_order]

# Formatear los porcentajes y decimales para mejor visualización
stats_completas['% reseñas con votos útiles'] = stats_completas['% reseñas con votos útiles'].round(2)
stats_completas = stats_completas.round(2)

# Mostrar resultados
print("Estadísticas de Votos Útiles (máximo 100 votos por reseña)")
display(stats_completas)

# 4. Top reseñas (versión compatible)
top_resenas = (
    df_filtrado
    .sort_values(['producto', 'useful_votes'], ascending=[True, False])
    .groupby('producto')
    .head(5)
    [['producto', 'useful_votes', 'text_length']]
    .reset_index(drop=True)
)

print("\n🔝 Top 5 reseñas más votadas por producto:")
display(top_resenas)

In [ ]:
sns.set_theme(style="darkgrid") 

# Filtrar datos (max 30 votos útiles)
df_filtrado = df_total[df_total['useful_votes'] <= 50]

plt.figure(figsize=(10,6))
sns.histplot(
    data=df_filtrado,
    x='useful_votes',
    color='darkgreen',  # Color único para todas las reseñas
    bins=30,
    edgecolor='white',  # Borde blanco para mejor definición
    alpha=0.6,
    #kde=True  # Añade línea de densidad
)

# Personalización
plt.title('Distribución de votos útiles (máximo 50 votos)', pad=20, fontsize=14)
plt.xlabel('Votos útiles', fontsize=12)
plt.ylabel('Cantidad de reseñas', fontsize=12)
plt.grid(True, axis='y', linestyle=':', alpha=0.3)

# Añadir anotaciones estadísticas
mean_votos = df_filtrado['useful_votes'].mean()
plt.axvline(mean_votos, color='red', linestyle='--', linewidth=1)
plt.text(mean_votos+0.5, plt.ylim()[1]*0.9, 
         f'Media: {mean_votos:.1f}', color='red')

plt.tight_layout()

plt.savefig("../outputs/visualizations/01_grafico_distribución_votos_utiles.png", dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
sns.set_theme(style="darkgrid") 
df_filtrado = df_total[df_total['useful_votes'] <= 30]

plt.figure(figsize=(10,6))
sns.histplot(
    data=df_filtrado,
    x='useful_votes',
    hue='producto',
    multiple='dodge',
    bins=30,
    edgecolor='black',
    alpha=0.7
)
plt.title('Distribución de votos útiles por producto (máximo 30 votos)')
plt.xlabel('Votos útiles')
plt.ylabel('Cantidad de reseñas')
plt.grid(True, axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.savefig("../outputs/visualizations/01_grafico_distribución_votos_utiles_por_producto.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 1. Generar tabla base de votos por mes y producto
votes_by_month_product = df_total.groupby(
    ['year_month', 'producto'], 
    observed=False
)['useful_votes'].agg(['sum', 'count', 'mean', 'max']).reset_index()
votes_by_month_product.columns = ['Mes', 'Producto', 'Total_Votos', 'Cantidad_Reseñas', 'Promedio_Votos', 'Max_Votos']

# 2. Calcular estadísticas agregadas por mes (totales)
votes_by_month = df_total.groupby('year_month')['useful_votes'].agg(['sum', 'count', 'mean', 'max']).reset_index()
votes_by_month.columns = ['Mes', 'Total_Votos', 'Cantidad_Reseñas', 'Promedio_Votos', 'Max_Votos']
votes_by_month['Producto'] = 'TOTAL'

# 3. Calcular estadísticas agregadas por producto (totales)
votes_by_product = df_total.groupby('producto')['useful_votes'].agg(['sum', 'count', 'mean', 'max']).reset_index()
votes_by_product.columns = ['Producto', 'Total_Votos', 'Cantidad_Reseñas', 'Promedio_Votos', 'Max_Votos']
votes_by_product['Mes'] = 'PERIODO COMPLETO'

# 4. Combinar todas las tablas
full_stats = pd.concat([
    votes_by_month_product,
    votes_by_month,
    votes_by_product
], ignore_index=True)

# 5. Calcular porcentaje de contribución
full_stats['Porcentaje_Total'] = (full_stats['Total_Votos'] / full_stats['Total_Votos'].sum()) * 100

# 6. Ordenar y formatear
full_stats = full_stats[[
    'Mes', 'Producto', 'Total_Votos', 'Porcentaje_Total', 
    'Cantidad_Reseñas', 'Promedio_Votos', 'Max_Votos'
]]

# Formatear números
full_stats['Porcentaje_Total'] = full_stats['Porcentaje_Total'].round(2)
full_stats['Promedio_Votos'] = full_stats['Promedio_Votos'].round(2)

# 7. Mostrar resultados
print("Datos Numéricos Detallados del Gráfico:")
display(full_stats)

# Guardar en CSV
stats_mensuales_reset.to_csv('estadisticas_mensuales_votos_utiles_por_producto.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 1. Preparación de datos
df_total['year_month'] = df_total['date'].dt.to_period('M').astype(str)  # Convertir a string para mejor visualización
df_total['producto'] = df_total['producto'].astype('category')

# 2. Agregación de votos útiles por mes y producto
votes_by_month_product = df_total.groupby(
    ['year_month', 'producto'], 
    observed=False
)['useful_votes'].sum().reset_index(name='total_votes')

# 5. Datos numéricos (opcional)
print("\nDatos Numéricos:")
display(votes_by_month_product.pivot(index='year_month', columns='producto', values='total_votes').fillna(0).astype(int))

# 3. Visualización mejorada
sns.set_theme(style="darkgrid") 
plt.figure(figsize=(14, 7))
ax = sns.barplot(
    x='year_month',
    y='total_votes',
    hue='producto',
    data=votes_by_month_product,
    #palette='viridis',  # Paleta amigable para daltónicos
    estimator=sum,
    errorbar=None
)

# 4. Personalización del gráfico
plt.title("Votos Útiles Acumulados por Mes y Producto\n", 
          fontsize=14, pad=20, fontweight='semibold')
plt.xlabel("\nMes", fontsize=12, labelpad=10)
plt.ylabel("Total de Votos Útiles\n", fontsize=12, labelpad=10)
plt.xticks(rotation=45, ha='right')

# 5. Mejoras adicionales
ax.grid(axis='y', linestyle='--', alpha=0.3)  # Grid horizontal discreto

# 7. Añadir etiquetas de valor (opcional para pocos datos)
for p in ax.patches:
    if p.get_height() > 0: 
        ax.annotate(
            f"{int(p.get_height())}",
            (p.get_x() + p.get_width() / 2., p.get_height()),
            ha='center',
            va='center',
            xytext=(0, 5),
            textcoords='offset points',
            fontsize=9
        )

plt.tight_layout()
plt.savefig("../outputs/visualizations/01_grafico_votos_utiles_por_mes_y_producto.png", dpi=300, bbox_inches='tight')
plt.show()

### 📊 Análisis Estadístico de Votos Útiles

#### 🔍 Diferencias entre productos

- El análisis general muestra un **engagement bajo** en términos de votos útiles: solo el **32.13%** de las reseñas los reciben.  
  - **El 75% de las reseñas** no obtiene ningún voto útil (**Q1 = 0** para ambos productos).  
- **Samsung A15** presenta una **mayor interacción**:
  - Más del **33.95%** de sus reseñas tienen votos útiles, frente al **29.88%** en Motorola G32.
  - También muestra una **media de votos por reseña más alta** (**2.90** vs **1.84**).
  - La **variabilidad** también es mayor en Samsung (**std = 2.90**) frente a Motorola (**std = 1.84**), indicando una distribución más dispersa (ver Gráfico 2).

#### 📈 Evolución temporal (Gráfico 3)

- **Motorola G32** concentra votos útiles entre diciembre de 2022 y fines de 2023, con **picos importantes en junio (552)** y **agosto (664)** de 2023.
  - A partir de 2024, la actividad se reduce drásticamente, con valores mensuales por debajo de 10 votos.
- **Samsung A15** comienza a recibir votos a partir de **abril de 2024**, con un **crecimiento explosivo** desde julio:
  - **Picos de 804 votos en julio**, **836 en septiembre** y una persistencia alta hasta principios de 2025 (**226 votos en enero**).
- Se observa así una **transición en el protagonismo de los productos**:
  - **Motorola domina 2023**, mientras que **Samsung lidera 2024 y 2025**.

#### ⚠️ Patrones destacados

1. **Distribución asimétrica** (Gráfico 1):
   - Predominio de reseñas con pocos votos (modo = 0), con una **cola larga hacia la derecha**.
2. **Complementariedad temporal**:
   - Motorola y Samsung se activan en **períodos casi excluyentes**, lo que sugiere ciclos diferenciados de interés del público.
3. **Presencia de outliers**:
   - **Samsung** alcanza más de **800 votos útiles** en varios meses (julio y septiembre 2024).
   - Contrasta con **meses de inactividad absoluta** de **Motorola en 2025**.


### b. Distribución de votos útiles por la longitud de la reseña

In [24]:
# Definir los rangos de longitud
bins = [0, 20, 40, 60, 80, 100, 150, 200]
labels = ["0–19", "20–39", "40–59", "60–79", "80–99", "100–149", "150–199"]

# Crear nueva columna con los bins
df_total['longitud_bin'] = pd.cut(df_total['text_length'], bins=bins, labels=labels, right=False)

In [ ]:
# Calcular estadísticas de votos útiles por longitud
stats_exactos = df_total.groupby('longitud_bin')['useful_votes'].agg([
    ('n', 'count'),
    ('media', 'mean'),
    ('mediana', 'median'),
    ('q1', lambda x: x.quantile(0.25)),
    ('q3', lambda x: x.quantile(0.75)),
    ('mínimo', 'min'),
    ('máximo', 'max'),
    ('std', 'std')
])
print(stats_exactos)
display(stats_exactos)

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(
    data=df_total,
    x='longitud_bin',
    y='useful_votes',
    showfliers=False,
    palette='pastel'
)

plt.title('Distribución de votos útiles según longitud de reseña', fontsize=14, pad=15)
plt.xlabel('Rango de palabras en la reseña', fontsize=12)
plt.ylabel('Votos útiles', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.savefig("../outputs/visualizations/01_grafico_votos_utiles_por_longitud.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
avg_votes = df_total.groupby('longitud_bin')['useful_votes'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x='longitud_bin', y='useful_votes', data=avg_votes, color='lightgreen')
plt.title('Promedio de votos útiles según longitud de reseña')
plt.xlabel('Rango de palabras en la reseña')
plt.ylabel('Votos útiles (promedio)')
plt.xticks(rotation=45)
plt.savefig("../outputs/visualizations/01_grafico_promedio_votos_utiles_por_longitud.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_total, x='text_length', y='useful_votes', hue='producto', alpha=0.9)
plt.title("Relación entre longitud de reseña y votos útiles")
plt.xlabel("Cantidad de palabras")
plt.ylabel("Votos útiles")
plt.legend(title="Producto")
plt.savefig("../outputs/visualizations/01_grafico_dispersion_votos_utiles_por_longitud.png", dpi=300, bbox_inches='tight')
plt.show()

### 📊 Análisis de Votos Útiles por Longitud de Reseña

#### 📦 Distribución por Rangos (Boxplot)

- Existe una **relación directa** entre longitud de reseña y cantidad de votos útiles, aunque con alta dispersión.  
- El rango **100–149 palabras** muestra:
  - **Mediana de 4 votos** (único tramo donde la mediana supera cero).
  - **Media más alta (50.3 votos)** y **máximo de 433 votos**.
- **Variabilidad creciente** a medida que crece la longitud:
  - En 100–149 palabras, la **desviación estándar alcanza 128.7**.

#### 📊 Promedio por Longitud (Barplot)

- Las reseñas de **0–19 palabras**, que dominan el conjunto (771 casos), apenas alcanzan **1.7 votos promedio**.
- Desde **40 palabras**, el promedio supera los **10 votos**.
- Las reseñas de más de **80 palabras** son solo el **2% del total**, pero concentran aproximadamente el **23% de los votos útiles**.

#### 🔁 Relación Longitud–Votos (Scatterplot)

- Se observan **dos patrones simultáneos**:
  1. Densidad alta de reseñas breves con 0–5 votos.
  2. Reseñas más largas con mayor dispersión y outliers destacados.
- La correlación es **no lineal**:
  - Tramos intermedios (100–149 palabras) rinden mejor que los más largos (>150), donde hay pocos casos y mayor dispersión.

#### ⚠️ Hallazgos Relevantes

- 🏆 **Rango óptimo**: 100–149 palabras combina volumen razonable, mediana positiva y máximos altos.  
- ⛔ **Rendimiento pobre**: el 77% de las reseñas de menos de 20 palabras no reciben ningún voto.  
- 🎯 **Efectos no deterministas**:
  - Algunas reseñas **cortas (20–39 palabras)** logran hasta **617 votos**, revelando que el contenido sigue siendo un factor clave.


### c. Distribución de votos útiles por calificación

In [ ]:
import pandas as pd

# 1. Calcular estadísticas básicas
stats = df_total.groupby('rating')['useful_votes'].describe(percentiles=[.25, .5, .75])
stats = stats.rename(columns={'50%': 'mediana'})

# 2. Calcular el total de votos por categoría
total_votos = df_total.groupby('rating')['useful_votes'].sum().rename('total_votos')

# 3. Calcular porcentaje de reseñas con votos
porc_con_votos = (df_total.groupby('rating')['useful_votes']
                         .apply(lambda x: (x > 0).mean() * 100)
                         .rename('%_con_votos'))

# 4. Unir todas las métricas
stats_completa = pd.concat([stats, total_votos, porc_con_votos], axis=1)

# 5. Reordenar columnas
column_order = [
    'count', '%_con_votos', 'total_votos', 'mean', 'std', 
    'min', '25%', 'mediana', '75%', 'max'
]
stats_completa = stats_completa[column_order]

# 6. Renombrar columnas para mejor visualización
stats_completa = stats_completa.rename(columns={
    'count': 'n_reseñas',
    'mean': 'media_votos',
    'std': 'desviación',
    'min': 'mínimo',
    'max': 'máximo',
    '25%': 'Q1',
    '75%': 'Q3'
})

# 7. Agregar fila de TOTALES
totales = pd.DataFrame({
    'n_reseñas': stats_completa['n_reseñas'].sum(),
    '%_con_votos': (df_total['useful_votes'] > 0).mean() * 100,
    'total_votos': stats_completa['total_votos'].sum(),
    'media_votos': df_total['useful_votes'].mean(),
    'desviación': df_total['useful_votes'].std(),
    'mínimo': df_total['useful_votes'].min(),
    'Q1': df_total['useful_votes'].quantile(0.25),
    'mediana': df_total['useful_votes'].median(),
    'Q3': df_total['useful_votes'].quantile(0.75),
    'máximo': df_total['useful_votes'].max()
}, index=['TOTAL'])

stats_completa = pd.concat([stats_completa, totales])

# 8. Formatear la salida
stats_completa = stats_completa.round({
    'media_votos': 2,
    'desviación': 2,
    '%_con_votos': 2
})

# Mostrar tabla
print("Estadísticas Completas de Votos Útiles por Rating")
display(stats_completa)

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(
    data=df_total, 
    x='rating', 
    y='useful_votes', 
    #hue='rating',
    palette='hls', 
    estimator='sum', 
    errorbar=None,
    legend=False
)
plt.title("Total de votos útiles por calificación", fontweight='bold')
plt.xlabel("Calificación (rating)")
plt.ylabel("Votos útiles")
plt.grid(axis='y', linestyle='--', alpha=0.7) 

for p in plt.gca().patches:
    plt.gca().annotate(
        f"{p.get_height():.1f}", 
        (p.get_x() + p.get_width() / 2, p.get_height()), 
        ha='center', 
        va='bottom'
    )
plt.savefig("../outputs/visualizations/01_grafico_total_votos_utiles_por_calificacion.png", dpi=300, bbox_inches='tight')
plt.show()

#### Gráfico de Distribución de Votos Útiles por Calificación

Visualiza cómo se distribuyen los votos útiles recibidos según la calificación otorgada (de 1 a 5 estrellas). Cada caja (boxplot) representa:
- La mediana (línea central)
- Los percentiles 25° y 75° (extremos de la caja)
- Los valores típicos (whiskers)
  
Se utiliza escala logarítmica porque:
1. **Presencia de outliers extremos**: Algunas reseñas tienen miles de votos mientras la mayoría tiene pocos
2. **Mejor legibilidad**: Comprime el rango visual para mostrar patrones en la mayoría de los datos
3. **Comparación efectiva**: Permite distinguir diferencias entre calificaciones que quedarían ocultas en escala lineal

*Nota*: El eje Y muestra el logaritmo del conteo de votos, donde cada incremento representa un múltiplo (ej: 1=10, 2=100, 3=1000 votos)

In [ ]:
plt.figure(figsize=(10, 6))

viridis_light = ["#f0f921", "#a0da39", "#4ac16d", "#1fa187", "#277f8e"]

# Crear el boxplot con escala logarítmica
boxplot = sns.boxplot(
    data=df_total,
    x='rating',
    y='useful_votes',
    hue='rating',
    palette=viridis_light,
    showfliers=False,
    width=0.7,
    linewidth=1.5,
    saturation=0.4
)

# Configuración de ejes y títulos
plt.yscale('log')
plt.title("Votos Útiles por Calificación (Escala Logarítmica)", 
          pad=20, fontsize=14, fontweight='bold')
plt.xlabel("Calificación", labelpad=15)  # Aumenté el padding
plt.ylabel("Votos útiles (log)", labelpad=15)
plt.grid(axis='y', linestyle='--', alpha=0.3)

# Leyenda mejor posicionada


# Ajuste de márgenes
plt.subplots_adjust(bottom=0.15)  # Aumenta espacio inferior

plt.tight_layout()
plt.savefig("../outputs/visualizations/01_grafico_votos_utiles_por_calificacion_log.png", dpi=300, bbox_inches='tight')
plt.show()

### ⭐ Distribución de Votos Útiles por Calificación

#### 📊 Total y Promedios por Calificación (Barplot)

Las reseñas con calificación **5 estrellas** concentran el mayor volumen de votos útiles, representando solo un 32.4% del total de reseñas, pero acumulando casi el 48% de los votos (2336 votos). Además, tienen una media notable de 6.66 votos por reseña, muy por encima de la media global de 4.49.

Las calificaciones intermedias, especialmente **3 y 4 estrellas**, muestran un comportamiento más heterogéneo. Mientras que las de 4 estrellas tienen un promedio relativamente alto (4.05 votos) pero con una gran dispersión (desviación estándar de 35.31) y frecuentes valores atípicos, las de 3 estrellas destacan por tener la media más baja (1.57 votos) y el menor porcentaje de reseñas con votos útiles (9.09%), aunque incluyen algunos outliers muy extremos (hasta 269 votos).

Por otro lado, las reseñas críticas de **1 y 2 estrellas** presentan un nivel de engagement alto y constante, destacándose las de 2 estrellas que superan incluso en porcentaje de reseñas con votos a las de 5 estrellas (56.9% vs. 45.0%).

#### 📦 Dispersión y Outliers (Boxplot en Escala Logarítmica)

La escala logarítmica del boxplot revela patrones que los promedios no capturan:  
- Las reseñas de 2 estrellas exhiben una distribución más consistente con mediana en 2 votos y rango intercuartílico (Q1-Q3) estrecho (0-4 votos).  
- Las reseñas de 5 estrellas tienen mediana en cero, indicando que más de la mitad no recibe votos, pero presentan una cola larga con outliers muy votados, hasta 433 votos, que elevan su media.  
- Las reseñas de 1 estrella también muestran un alto porcentaje de reseñas con votos útiles (70.7%) y una distribución menos dispersa que las de 5 estrellas.

Los valores atípicos más elevados corresponden a reseñas con 4 estrellas (617 votos), 5 estrellas (433 votos) y 3 estrellas (269 votos), lo que evidencia la presencia de "reseñas virales" con gran impacto independientemente de su calificación.

#### ⚠️ Patrones Relevantes y Conclusiones

- El engagement con votos útiles es claramente **polarizado**, concentrándose en las reseñas más extremas (1 y 5 estrellas), aunque las negativas (1 y 2 estrellas) tienen un mayor porcentaje de reseñas con votos, lo que sugiere un efecto de mayor reacción ante opiniones críticas.  
- Las reseñas de 3 estrellas, en cambio, parecen ser las menos valoradas por la comunidad, a pesar de incluir algunos casos excepcionales con mucha visibilidad.  
- La alta variabilidad en los votos de 4 y 5 estrellas indica que no todas las reseñas en estos grupos tienen el mismo impacto, y que una minoría puede ser extremadamente influyente.  
- En conjunto, estos patrones reflejan una dinámica de votación donde la intensidad emocional (muy positiva o muy negativa) y la viralidad influyen decisivamente en la distribución de votos útiles.


### Matriz de Correlación (Heatmap) entre calificación, votos útiles y longitud del texto

Este gráfico muestra las relaciones estadísticas entre tres variables clave de las reseñas:

- **Calificación**: Puntuación asignada por el usuario
- **Votos útiles**: Cantidad de usuarios que marcaron la reseña como útil
- **Longitud del texto**: Número de palabras en la reseña

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Configuración general
plt.figure(figsize=(9, 7))
sns.set(style="white", font_scale=1.1)

# Datos (asumiendo que ya tienes corr_matrix)
# Si no, puedes generarla así:
nombre_columnas = {
    'rating': 'Calificación',
    'useful_votes': 'Votos útiles',
    'text_length': 'Longitud del texto'
}
df_plot = df_total[['rating','useful_votes','text_length']].rename(columns=nombre_columnas)
corr_matrix = df_plot.corr()

# Creación del heatmap con paleta 'vlag'
ax = sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap='vlag',
    center=0,
    vmin=-1,
    vmax=1,
    linewidths=.5,
    linecolor='gray',
    square=True,
    cbar_kws={'shrink': 0.75, 'label': 'Coeficiente de Correlación'}
)

# 1. Mejora de etiquetas y título
ax.set_title('Relación entre Variables de Comentarios\n', pad=20, fontsize=14, fontweight='bold')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_yticklabels(ax.get_yticklabels(), rotation=0)

# 2. Destacar valores importantes (correlaciones > 0.2)
for i in range(len(corr_matrix)):
    for j in range(len(corr_matrix)):
        if abs(corr_matrix.iloc[i,j]) > 0.2 and i != j:  # Excluye la diagonal
            ax.text(j+0.5, i+0.5, f"{corr_matrix.iloc[i,j]:.2f}", 
                   ha='center', va='center', 
                   fontweight='bold', fontsize=12,
                   bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=2))

# 3. Líneas de división mejoradas
ax.hlines([1, 2], *ax.get_xlim(), colors='gray', linestyles='dashed', linewidth=0.8, alpha=0.7)
ax.vlines([1, 2], *ax.get_ylim(), colors='gray', linestyles='dashed', linewidth=0.8, alpha=0.7)

# 4. Leyenda explicativa (agregado 1)
plt.figtext(0.5, -0.15, 
            "Valores cercanos a +1: fuerte correlación positiva\n"
            "Valores cercanos a -1: fuerte correlación negativa\n"
            f"Correlaciones significativas (|r| > 0.2) destacadas en negrita", 
            ha='center', fontsize=10, bbox=dict(facecolor='whitesmoke', alpha=0.5, boxstyle='round'))

# Ajuste final
plt.tight_layout()
plt.savefig("../outputs/visualizations/01_grafico_heatmap_votos_utiles_longitud_calificacion.png", dpi=300, bbox_inches='tight')
plt.show()

### Tabla de correlaciones entre calificación, votos útiles y longitud del texto

La siguiente tabla presenta los coeficientes de correlación de Pearson y Spearman entre las variables de calificación (rating), cantidad de votos útiles (useful_votes) y longitud del texto de la reseña (text_length). Los coeficientes de Pearson informan sobre la relación lineal, mientras que los coeficientes de Spearman evalúan la relación monótona entre las variables. Se reporta el coeficiente de correlación seguido del valor p asociado.

In [ ]:
from scipy.stats import pearsonr, spearmanr
import pandas as pd

# Pearson (relación lineal)
pearson_rating_votes = pearsonr(df_total['rating'], df_total['useful_votes'])
pearson_rating_length = pearsonr(df_total['rating'], df_total['text_length'])
pearson_votes_length = pearsonr(df_total['useful_votes'], df_total['text_length'])

# Spearman (relación monótona)
spearman_rating_votes = spearmanr(df_total['rating'], df_total['useful_votes'])
spearman_rating_length = spearmanr(df_total['rating'], df_total['text_length'])
spearman_votes_length = spearmanr(df_total['useful_votes'], df_total['text_length'])

# Crear dataframe con resultados
results = pd.DataFrame({
    'Par de variables': [
        'Rating vs Votes',
        'Rating vs Length',
        'Votes vs Length'
    ],
    'Pearson r': [
        f"{pearson_rating_votes[0]:.3f} (p={pearson_rating_votes[1]:.3f})",
        f"{pearson_rating_length[0]:.3f} (p={pearson_rating_length[1]:.3f})",
        f"{pearson_votes_length[0]:.3f} (p={pearson_votes_length[1]:.3f})"
    ],
    'Spearman rho': [
        f"{spearman_rating_votes[0]:.3f} (p={spearman_rating_votes[1]:.3f})",
        f"{spearman_rating_length[0]:.3f} (p={spearman_rating_length[1]:.3f})",
        f"{spearman_votes_length[0]:.3f} (p={spearman_votes_length[1]:.3f})"
    ]
})

# Imprimir tabla en formato limpio
print(results.to_string(index=False))


### 📉 5. Estudio de caso: Análisis de Crisis Reputacional

El siguiente gráfico combina dos métricas clave para monitorear la percepción pública de un producto a lo largo del tiempo: el **rating promedio mensual** (eje izquierdo) y la **mediana de votos útiles por comentario** (eje derecho). Ambas variables fueron calculadas como estadísticos de tendencia central para capturar el **tono cualitativo** de las valoraciones y su **recepción entre usuarios**, independientemente del volumen absoluto de reseñas.

Este enfoque resulta particularmente útil para detectar fenómenos de **crisis reputacional**, ya que el **rating promedio expresa con mayor sensibilidad las variaciones en la calidad percibida**, mientras que la **mediana de votos útiles** (menos sensible a valores extremos) refleja el nivel típico de engagement de la comunidad con las reseñas.

#### Patrón Observado

Los datos revelan un patrón característico de crisis reputacional:

**Fase de Deterioro (Julio-Agosto 2024)**
- **Julio**: Rating alto (4.18) con nivel moderado de votos útiles (8.0)
- **Agosto**: Primera caída significativa del rating (3.62) con descenso en engagement (2.0)

**Fase Crítica (Septiembre 2024)**
- **Punto de inflexión**: Rating se mantiene bajo (3.58) pero los votos útiles alcanzan su máximo (10.0)
- Esta convergencia sugiere que las reseñas críticas **fueron valoradas como especialmente útiles** por la comunidad

**Profundización de la Crisis (Octubre-Noviembre 2024)**
- **Mínimo histórico**: Rating desciende a 2.70 en noviembre
- Los votos útiles se mantienen en niveles bajos pero estables (1.0-2.0)

**Recuperación Gradual (2025)**
- **Mejora sostenida**: Rating se recupera progresivamente hasta 4.18 en abril
- **Normalización del engagement**: Votos útiles descienden a niveles mínimos (0.0)

#### Interpretación

La secuencia temporal sugiere que **septiembre de 2024** fue el momento crítico donde la comunidad de usuarios mostró mayor interés en compartir y valorar información sobre las deficiencias del producto. La posterior recuperación del rating, acompañada por la normalización de los votos útiles, indica una resolución gradual de los problemas subyacentes.

> 💡 *El uso de mediana para votos útiles, en lugar de promedio, permite identificar mejor este tipo de fenómenos cualitativos al reducir la influencia de valores extremos ocasionales.*


In [ ]:
# Código para generar df_metrics a partir de df_total

# 1. Filtrar solo productos Samsung
df_samsung = df_total[df_total['producto'].str.contains('Samsung', case=False, na=False)].copy()

# 2. Convertir la columna date a datetime si no lo está ya
df_samsung['date'] = pd.to_datetime(df_samsung['date'])

# 3. Filtrar el período desde julio 2024 hasta 2025
start_date = '2024-07-01'
end_date = '2025-12-31'  # Ajusta según necesites
df_samsung_filtered = df_samsung[
    (df_samsung['date'] >= start_date) & 
    (df_samsung['date'] <= end_date)
].copy()

# 4. Crear una columna year_month_str en formato YYYY-MM para agrupación
df_samsung_filtered['year_month_str'] = df_samsung_filtered['date'].dt.strftime('%Y-%m')

# 5. Agrupar por year_month_str y calcular métricas
# Usar promedio para rating y mediana para useful_votes (menos sensible a outliers)
df_metrics = df_samsung_filtered.groupby('year_month_str').agg({
    'rating': 'mean',
    'useful_votes': 'median'  # Mediana para evitar distorsión por outliers
}).reset_index()

# 6. Redondear los valores para mejor presentación
df_metrics['rating'] = df_metrics['rating'].round(2)
df_metrics['useful_votes'] = df_metrics['useful_votes'].round(1)

# 7. Ordenar por fecha para que el gráfico se vea correctamente
df_metrics = df_metrics.sort_values('year_month_str').reset_index(drop=True)

print(f"Shape de df_metrics: {df_metrics.shape}")
print(df_metrics)
print(f"\nRango de fechas: {df_metrics['year_month_str'].min()} a {df_metrics['year_month_str'].max()}")

In [ ]:
sns.set_theme(style="darkgrid") 
plt.rcParams.update({
    'font.family': 'DejaVu Sans',
    'axes.titlesize': 14,
    'axes.titleweight': 'bold',
    'axes.titlepad': 25
})

df_plot = df_metrics.dropna(subset=['useful_votes']).copy()
df_plot.reset_index(drop=True, inplace=True)

fig, ax1 = plt.subplots(figsize=(14, 8.5)) 
plt.subplots_adjust(top=0.75, right=0.85, left=0.1, hspace=0.4)

line = ax1.plot(df_plot['year_month_str'], 
                df_plot['rating'],
                color='#1F77B4',
                marker='o',
                markersize=9,
                linewidth=2.5,
                label='Rating Promedio (1-5 estrellas)',
                zorder=3)

ax1.set_ylabel('Rating Promedio', fontsize=12, color='#1F77B4')
ax1.set_ylim(1, 6.0) 
ax1.tick_params(axis='y', colors='#1F77B4')


ax2 = ax1.twinx()
bars = ax2.bar(df_plot['year_month_str'],
               df_plot['useful_votes'],
               color='#FF7F0E',
               alpha=0.7,
               width=0.4,
               label='Votos útiles promedio',
               zorder=2)

ax2.set_ylabel('Votos útiles promedio', fontsize=12, color='#FF7F0E')
ax2.set_ylim(0, max(df_plot['useful_votes']) * 1.25) 

ax1.set_xlabel('Mes-Año', fontsize=12)
plt.xticks(rotation=45, ha='right')

for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2.,
             height + max(df_plot['useful_votes'])*0.03,
             f'{height:.1f}',
             ha='center',
             va='bottom',
             color='#FF7F0E',
             fontsize=10,
             bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.2'))

handles1, labels1 = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
plt.legend(handles1+handles2, labels1+labels2,
           loc='upper center',
           bbox_to_anchor=(0.5, 1.1), 
           ncol=2,
           frameon=True,
           fontsize=11)

crisis_month = '2024-08'
if crisis_month in df_plot['year_month_str'].values:
    idx = list(df_plot['year_month_str']).index(crisis_month)
    max_votes = max(df_plot['useful_votes'])
    
    arrow_y = max_votes * 0.95 
    
    text_y = max_votes * 1.15 
    
    ax2.annotate('CRISIS REPUTACIONAL',
                xy=(idx, arrow_y), 
                xytext=(idx, text_y), 
                ha='center',
                va='center',
                fontsize=12,
                fontweight='bold',
                color='red',
                bbox=dict(boxstyle='round,pad=0.3', 
                         facecolor='white', 
                         edgecolor='red', 
                         alpha=0.9),
                arrowprops=dict(
                    arrowstyle='->', 
                    color='red', 
                    linewidth=1.5,
                    connectionstyle="arc3,rad=0.2" 
                ),
                zorder=4)
    
plt.savefig("../outputs/visualizations/01_grafico_crisis_reputacional_samsung.png", dpi=300, bbox_inches='tight')

plt.show()